# Solutions to exercises in the `ray-core` Lessons

First, import everything we'll need and start Ray:

In [2]:
import ray, time, sys
import numpy as np
sys.path.append('../..')
from util.printing import pnd, pd

In [2]:
ray.init(ignore_reinit_error=True)

2020-04-06 13:44:01,589	INFO resource_spec.py:204 -- Starting Ray with 4.0 GiB memory available for workers and up to 2.02 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-06 13:44:01,947	INFO services.py:1146 -- View the Ray dashboard at localhost:8267


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:15803',
 'object_store_address': '/tmp/ray/session_2020-04-06_13-44-01_581721_30006/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-06_13-44-01_581721_30006/sockets/raylet',
 'webui_url': 'localhost:8267',
 'session_dir': '/tmp/ray/session_2020-04-06_13-44-01_581721_30006'}

## Exercise 1 in 02-DataParallelism-Part1

You were asked to convert the regular Python code to Ray code. Here are the three cells appropriately modified.

First, we need the appropriate imports and `ray.init()`.

In [3]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

In [4]:
start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = ray.get(ids)
duration = time.time() - start

In [5]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 

## Exercise 2 in 03-DataParallelism-Part2

You were asked to use `ray.wait()` with a shorter timeout, `2.5` seconds. First we need to define the remote functions we used in that lesson:

In [6]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

In [10]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) >= 2 else 1   # See discussion in the next cell
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=0.5)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    for array in ray.get(ready_ids):
        print(f'{array.size}: {array}')
        
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  1.012)
0: []
Returned   1 completed tasks. (elapsed time:  2.022)
10: [-0.20901075 -0.72949182 -0.28642699  0.22743713  3.95596187 -0.88017923
 -3.34731201  0.43302113 -0.47698767  1.78129132]
Returned   0 completed tasks. (elapsed time:  3.032)
Returned   1 completed tasks. (elapsed time:  4.033)
20: [-1.13928061 -3.50407667  2.38028826  0.32258259 -1.96872189 -2.06495504
 -0.47164318  2.3409099   0.21522455  5.64403487  0.57652634  0.92022058
 -0.20882429 -0.2094916  -1.90197131  0.90725176 -0.42482058 -1.60838073
  0.65295721 -1.30397866]
Returned   0 completed tasks. (elapsed time:  5.040)
Returned   1 completed tasks. (elapsed time:  6.049)
30: [ 0.27803524  0.83967401  1.09990838  2.63662949 -1.36933238 -2.82048063
 -0.93562439  1.90033255 -1.80210399  1.17175202  1.41702166  3.68647442
 -2.11658026 -1.67363274 -1.78038492  0.80202995 -0.56110565 -1.10746302
  1.36555112 -0.6232434   0.77354945  2.06223836 -0.70719446 -2.05362656
 -1.

For a timeout of `2.5` seconds, the second call to `ray.wait()` times out before two tasks finish, so it only returns one completed task. Why did the third and last iteration not time out? (That is, they both successfully returned two items.) It's because all the tasks were running in parallel so they had time to finish. If you use a shorter timeout, you'll see more time outs, where zero or one items are returned. 

Try `1.5` seconds, where all but one iteration times out and returns one item. The first iteration returns two items.
Try `0.5` seconds, where you'll get several iterations that time out and return zero items, while the all the other iterations time out and returns one item.